In [7]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/anura/anaconda3/Library/bin/graphviz'
import sys
import networkx as nx
import numpy as np
import random
import pickle
import sys
from numba import jit
import elfi
import sklearn as sk
import scipy
from scipy.spatial import distance
import pandas as pd
import sys
print(sys.version)

3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]


In [8]:
seed = 20170530  # this will be separately given to ELFI
np.random.seed(seed)

networktype = 'pref' #pref, smallworld, grid, ER, korea1, korea2, ckm


In [11]:
#parameters of the script
path = '../data/icpsr/DS0001/paluck-edgelist.csv'
edgelist = pd.read_csv(path)
G = nx.from_pandas_edgelist(edgelist, source='ID', target='PEERID')
print(nx.info(G))
print(nx.number_of_nodes(G))
print(f'connected?\t{nx.is_connected(G)}')
print(f'# of connected components:\t{nx.number_connected_components(G)}')

components = nx.connected_components(G)
sglist = [G.subgraph(c) for c in nx.connected_components(G)]

gmat = []
for g in sglist:
    gmat.append(nx.to_numpy_matrix(g, dtype=np.float))

Name: 
Type: Graph
Number of nodes: 23882
Number of edges: 132054
Average degree:  11.0589
23882
connected?	False
# of connected components:	57


In [24]:
seeds = pd.read_csv("../data/icpsr/DS0001/paluck-seed.csv")
seeds['ID'] = ((seeds['SCHIDW2'] * 1000) + pd.to_numeric(seeds['ID'], errors='coerce'))
es = pd.read_csv("../data/icpsr/DS0001/paluck-endstate.csv")
es['ID'] = ((es['SCHIDW2'] * 1000) + pd.to_numeric(es['ID'], errors='coerce'))

 /home/gary/.conda/envs/elfi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning:Columns (238,242,244,245,246,247,248,249,382) have mixed types.Specify dtype option on import or set low_memory=False.


In [29]:
count = 0
dfl = []
for ng in range(nx.number_connected_components(G)):
    #n,a = getGraphFromEdgelist(path)
    net_s = sglist[ng]
    adjMat_s = gmat[ng]
    count += 1
    inf = nx.info(net_s, n = None)
    n_nodes = nx.number_of_nodes(net_s)
    n_edges = nx.number_of_edges(net_s)
    node_degree = nx.degree(net_s)
    n_degree = [deg for (node,deg) in node_degree]
    avg_degree = sum(list(n_degree))/len(list(n_degree))
    
    nAgents = nx.number_of_nodes(net_s)
    agents = np.zeros((1, nAgents))
    nodes = nx.nodes(net_s)
    seed_nodes = []
    inx = []
    count_seed = 0

    es_nodes = []
    es_inx = []
    count_es = 0
    num_es = 0

    
    for n in nodes:
        if n in seeds['ID'].tolist():
            seed_nodes.append(n)
            inx.append(count)
            count_seed += 1

        if n in es['ID'].tolist():
            es_nodes.append(n)
            es_inx.append(count_es)
            num_es += 1
        count_es += 1
        
    agents[:,inx] = 1

    p_activation = num_es/count_es
    
    dfl.append([count, n_nodes, n_edges, avg_degree, count_seed, num_es, p_activation])
    
    
dfs = pd.DataFrame(dfl, columns = ["graph_id", "n_nodes", "n_edges", "average_degree", 
                                   "no._of_seeds", "no._of_final_activated_nodes", "percentage_activation"])
dfs.to_csv("../data/summary_stats/graph_summaries.csv", index = False)

In [30]:
dfs

,graph_id,n_nodes,n_edges,average_degree,no._of_seeds,no._of_final_activated_nodes,percentage_activation
0,1,272,1481,10.889706,19,124,0.455882
1,2,518,2866,11.065637,0,450,0.868726
2,3,137,721,10.525547,15,45,0.328467
3,4,113,569,10.070796,0,104,0.920354
4,5,459,2651,11.551198,24,264,0.575163
5,6,618,3516,11.378641,0,577,0.933657
6,7,627,3328,10.615630,0,561,0.894737
7,8,394,2080,10.558376,20,212,0.538071
8,9,316,1833,11.601266,24,150,0.474684
9,10,479,2548,10.638831,0,449,0.937370


       SCHID      UID  SCHTREAT  SCHRB       ID  TREAT  STRB   SID1   SID3  \
0          1   100002         1    3.0   1002.0      2     4   5108   5108   
1          1   100003         1    3.0   1003.0      0     0   5100   5100   
2          1   100008         1    3.0   1008.0      0     0   5129   5129   
3          1   100010         1    3.0   1010.0      0     0   5089   5089   
4          1   100013         1    3.0   1013.0      1     4   5126   5126   
...      ...      ...       ...    ...      ...    ...   ...    ...    ...   
17533     60  6000669         1   11.0  60669.0      0     0  14095  14095   
17534     60  6000670         1   11.0  60670.0    -98   -98  13897  13897   
17535     60  6000671         1   11.0  60671.0    -98   -98  14054  14054   
17536     60  6000672         1   11.0  60672.0    -98   -98  13985  13985   
17537     60  6000674         1   11.0  60674.0    -98   -98                 

        SID4  ... AM_F PI_M PI_F  MU_M  MU_F  ROW_TOTAL  FREE_L

In [31]:
set_objective

'/usr/local/anaconda/lib/python3.6/site-packages/elfi/model/elfi_model.py'

In [30]:
lines = inspect.getsource(elfi.Distance)
print(lines)

class Distance(Discrepancy):
    """A convenience class for the discrepancy node."""

    def __init__(self, distance, *summaries, **kwargs):
        """Initialize a distance node of an ELFI graph.

        This class contains many common distance implementations through scipy.

        Parameters
        ----------
        distance : str, callable
            If string it must be a valid metric from `scipy.spatial.distance.cdist`.

            Is a callable, the signature must be `distance(X, Y)`, where X is a n x m
            array containing n simulated values (summaries) in rows and Y is a 1 x m array
            that contains the observed values (summaries). The callable should return
            a vector of distances between the simulated summaries and the observed
            summaries.
        *summaries
            Summary nodes of the model.
        **kwargs
            Additional parameters may be required depending on the chosen distance.
            See the scipy document

In [3]:
import inspect
inspect.getfile(elfi.SMC)

'/home/gary/.conda/envs/elfi/lib/python3.7/site-packages/elfi/methods/parameter_inference.py'